In [1]:
import pandas as pd 
import csv
import spacy
import pt_core_news_sm
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from geopy import geocoders  
import string
import time

In [200]:
df = pd.read_csv(r'Vagas+Catho+Gupy+Indeed.csv')

In [201]:
df.head()

,Cidade,Competências,Descrição_da_empresa,Empresa,Estado,Nome_da_vaga,Termo_pesquisado
0,brasilia,o cientista de dados ou data scientist e consi...,NaN,NaN,DF,vaga de cientista de dados em brasilia - df,cientista de dados
1,londrina,"utilizar modelos matematicos e estatisticos, p...",NaN,NaN,PR,vaga de cientista de dados em londrina - pr,cientista de dados
2,rio de janeiro,interpretacao de tendencias. refino de base de...,NaN,NaN,RJ,vaga de data scientist junior/ cientista de da...,cientista de dados
3,rio de janeiro,"modelar problemas complexos, identificar oport...",NaN,NaN,RJ,vaga de data scientist senior/ cientista de da...,cientista de dados
4,rio de janeiro,realizar projetos sprint de desenvolvimento e ...,NaN,NaN,RJ,vaga de cientista de dados (aut) em rio de jan...,cientista de dados


In [276]:
df_cidades = df.copy()

In [277]:
df_cidades.head()

,Cidade,Competências,Descrição_da_empresa,Empresa,Estado,Nome_da_vaga,Termo_pesquisado,Latitude,Longitude
0,brasilia,o cientista de dados ou data scientist e consi...,NaN,NaN,DF,vaga de cientista de dados em brasilia - df,cientista de dados,-10.333333,-53.200000
1,londrina,"utilizar modelos matematicos e estatisticos, p...",NaN,NaN,PR,vaga de cientista de dados em londrina - pr,cientista de dados,-23.311288,-51.159502
2,rio de janeiro,interpretacao de tendencias. refino de base de...,NaN,NaN,RJ,vaga de data scientist junior/ cientista de da...,cientista de dados,-22.911014,-43.209373
3,rio de janeiro,"modelar problemas complexos, identificar oport...",NaN,NaN,RJ,vaga de data scientist senior/ cientista de da...,cientista de dados,-22.911014,-43.209373
4,rio de janeiro,realizar projetos sprint de desenvolvimento e ...,NaN,NaN,RJ,vaga de cientista de dados (aut) em rio de jan...,cientista de dados,-22.911014,-43.209373


In [278]:
columns = ['Competências', 'Descrição_da_empresa', 'Empresa', 'Nome_da_vaga', 'Termo_pesquisado']
df_cidades.drop(columns, inplace=True, axis=1)

In [279]:
df_cidades

,Cidade,Estado,Latitude,Longitude
0,brasilia,DF,-10.333333,-53.200000
1,londrina,PR,-23.311288,-51.159502
2,rio de janeiro,RJ,-22.911014,-43.209373
3,rio de janeiro,RJ,-22.911014,-43.209373
4,rio de janeiro,RJ,-22.911014,-43.209373
5,rio de janeiro,RJ,-22.911014,-43.209373
6,rio de janeiro,RJ,-22.911014,-43.209373
7,rio de janeiro,RJ,-22.911014,-43.209373
8,porto alegre,RS,-30.032500,-51.230377
9,blumenau,SC,-26.919557,-49.065802


In [280]:
#data.groupby(['month', 'item'])['date'].count()
df_cidades.groupby(['Cidade', 'Estado']).count()

,,Latitude,Longitude
Cidade,Estado,,
abelardo luz,SC,1,1
aguas mornas,SC,1,1
ananindeua,PA,2,2
aparecida de goiania,GO,3,3
apiai,SP,2,2
aquiraz,CE,1,1
aracariguama,SP,1,1
arapongas,PR,1,1
araraquara,SP,2,2


In [281]:
df_cidades.drop_duplicates(subset ="Cidade", inplace = True) 

In [282]:
df_cidades = df_cidades.reset_index()

In [283]:
df_cidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 5 columns):
index        163 non-null int64
Cidade       163 non-null object
Estado       140 non-null object
Latitude     163 non-null float64
Longitude    163 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 6.4+ KB


In [284]:
df_cidades

,index,Cidade,Estado,Latitude,Longitude
0,0,brasilia,DF,-10.333333,-53.200000
1,1,londrina,PR,-23.311288,-51.159502
2,2,rio de janeiro,RJ,-22.911014,-43.209373
3,8,porto alegre,RS,-30.032500,-51.230377
4,9,blumenau,SC,-26.919557,-49.065802
5,10,florianopolis,SC,-27.597300,-48.549610
6,12,sao caetano do sul,SP,-23.619592,-46.568832
7,13,sao paulo,SP,-23.550651,-46.633382
8,33,campinas,SP,-22.905560,-47.060830
9,34,ribeirao preto,SP,-21.178333,-47.806667


In [285]:
df_cidades['Cidade'][0]

'brasilia'

In [297]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='myapplication')

for i in range(141,len(df_cidades['Cidade'])):
    local = df_cidades['Cidade'][i]
    location = geolocator.geocode(local, exactly_one=False)[0]
    print(local)
    print(location.latitude)
    print(location.longitude)
    df.loc[df['Cidade'] == df_cidades['Cidade'][i], 'Latitude'] = location.latitude
    df.loc[df['Cidade'] == df_cidades['Cidade'][i], 'Longitude'] = location.longitude
    df_cidades.loc[df_cidades['Cidade'] == df_cidades['Cidade'][i], 'Latitude'] = location.latitude
    df_cidades.loc[df_cidades['Cidade'] == df_cidades['Cidade'][i], 'Longitude'] = location.longitude
    time.sleep( 4 )

marilia
-22.217108
-49.95006
americana
-22.7392463
-47.3306032
maracanau
-3.8779438
-38.6247697
hortolandia
-22.85833
-47.22
rondonopolis
-16.4603533
-54.6397099
itajai
-26.9117879
-48.6669852
passo fundo
-28.2616138
-52.4089048
barra mansa
-22.5394843
-44.1735129
pindamonhangaba
-22.9340165
-45.462938
ipatinga
-19.4777807
-42.5270802
parauapebas
-6.0696846
-49.8903283
aracaju
-10.9162061
-37.0774655
campos dos goytacazes
-21.7558619
-41.3326895
uberaba
-19.750833
-47.936666
macae
-22.3712958
-41.7866922
rio verde
-17.7921255
-50.9191219
paranagua
-25.5121904
-48.5251854
queimados
-22.7156978
-43.555754
sao pa
40.9699889
-77.7278831
londrina 
-23.3112878
-51.1595023
valinhos
-22.97056
-46.99583
br
25.6440845
85.906508


In [292]:
df_cidades

,index,Cidade,Estado,Latitude,Longitude
0,0,brasilia,DF,-15.793404,-47.882317
1,1,londrina,PR,-23.311288,-51.159502
2,2,rio de janeiro,RJ,-22.911014,-43.209373
3,8,porto alegre,RS,-30.032500,-51.230377
4,9,blumenau,SC,-26.919557,-49.065802
5,10,florianopolis,SC,-27.597300,-48.549610
6,12,sao caetano do sul,SP,-23.619592,-46.568832
7,13,sao paulo,SP,-23.550651,-46.633382
8,33,campinas,SP,-22.905560,-47.060830
9,34,ribeirao preto,SP,-21.178333,-47.806667


In [293]:
df

,Cidade,Competências,Descrição_da_empresa,Empresa,Estado,Nome_da_vaga,Termo_pesquisado,Latitude,Longitude
0,brasilia,o cientista de dados ou data scientist e consi...,NaN,NaN,DF,vaga de cientista de dados em brasilia - df,cientista de dados,-15.793404,-47.882317
1,londrina,"utilizar modelos matematicos e estatisticos, p...",NaN,NaN,PR,vaga de cientista de dados em londrina - pr,cientista de dados,-23.311288,-51.159502
2,rio de janeiro,interpretacao de tendencias. refino de base de...,NaN,NaN,RJ,vaga de data scientist junior/ cientista de da...,cientista de dados,-22.911014,-43.209373
3,rio de janeiro,"modelar problemas complexos, identificar oport...",NaN,NaN,RJ,vaga de data scientist senior/ cientista de da...,cientista de dados,-22.911014,-43.209373
4,rio de janeiro,realizar projetos sprint de desenvolvimento e ...,NaN,NaN,RJ,vaga de cientista de dados (aut) em rio de jan...,cientista de dados,-22.911014,-43.209373
5,rio de janeiro,"desenvolvimento de software, ajudar na constru...",NaN,NaN,RJ,vaga de cientista de dados em rio de janeiro - rj,cientista de dados,-22.911014,-43.209373
6,rio de janeiro,"sprints adicionais / avulsos de analytics, mac...",NaN,NaN,RJ,vaga de cientista de dados - ml em rio de jane...,cientista de dados,-22.911014,-43.209373
7,rio de janeiro,"modelar problemas complexos, identificar oport...",NaN,NaN,RJ,vaga de cientista de dados em rio de janeiro - rj,cientista de dados,-22.911014,-43.209373
8,porto alegre,"empresa referencia em data driven marketing,...",NaN,NaN,RS,vaga de cientista de dados em porto alegre - rs,cientista de dados,-30.032500,-51.230377
9,blumenau,"sera sua responsabilidade o levantamento, anal...",NaN,NaN,SC,vaga de cientista de dados em blumenau - sc,cientista de dados,-26.919557,-49.065802


In [298]:
df.to_csv('Vagas+Catho+Gupy+Indeed+LatitudeLongitude.csv')